In [1]:
import collections
import re


In [ ]:

import re
import requests

def download_time_machine():
    """独立下载并处理时间机器数据集"""
    url = "https://www.gutenberg.org/files/35/35-0.txt"
    response = requests.get(url)
    lines = response.text.split('\n')
    print(lines[0])  # 输出首行
    print(lines[-100]) # 输出第11行
    # 跳过古登堡计划的元数据部分

    
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() 
           for line in lines if line.strip()]

lines = download_time_machine()
print(f'文本总行数: {len(lines)}')
print(lines[0])  # 输出首行
print(lines[10]) # 输出第11行


*** START OF THE PROJECT GUTENBERG EBOOK 35 ***
told him he was suffering from overwork, at which he laughed hugely. I


ValueError: '*** START OF THE PROJECT GUTENBERG EBOOK 35 ***' is not in list